# Tutorial


This tutoriel should help you to better understand how cafram works.

## Howto - Create basic config

Let's imagine a basic application that create a list of file from and apply a list of filters to them. Let's start by modelize how we would control this application, and we come with this configuration:

In [1]:
import yaml
import json
from pprint import pprint

yaml_config = """
version: 1
config:
  create_file: True
  backups: 3
  backup_prefix: null
  nested_dict:
    key1: val1
    key2: val2
  
files:
  - name: hello
    filters:
      - author
  - name: world
  
filters:
  content:
    prepend: "Added at first"
    append: "Added at last"
  author:
    name: MyName
    date: True
"""

# Load the yaml config python variable
config = yaml.safe_load(yaml_config)


Then our configuration looks like:

In [2]:
print (json.dumps(config, indent=2))


{
  "version": 1,
  "config": {
    "create_file": true,
    "backups": 3,
    "backup_prefix": null,
    "nested_dict": {
      "key1": "val1",
      "key2": "val2"
    }
  },
  "files": [
    {
      "name": "hello",
      "filters": [
        "author"
      ]
    },
    {
      "name": "world"
    }
  ],
  "filters": {
    "content": {
      "prepend": "Added at first",
      "append": "Added at last"
    },
    "author": {
      "name": "MyName",
      "date": true
    }
  }
}


## Learn - The three object type


The thumb of rule is the following:


* ConfDict
    * Usage: For dictionnary store
    * Children: Yes
    * Type: must be a dict
* ConfAttr
    * Like ConfDict but provides the attribute acces to nested object
* ConfList
    * Usage: For list store
    * Children: Yes
    * Type: must be a list
* ConfVal:
    * Usage: Any atomic item
    * Children: No
    * Type: Anything json serializable
* Native Type: None,Bool,Int,Str,Dict,List
    * Usage: Raw json compatible data
    * Children: No
    * Type: Anything json serializable
    * Does not provide anything other default object methods.
    
    
You always have to thing on what is possible in json or not, and adapt to your application structure.

Rule of thumbs: If not serializable, don't do it. I specifically think about OrderedDicts.

## Howto - Make magic

In [3]:
from cafram.nodes_conf import NodeAuto

app0 = NodeAuto(ident="app", payload=config, autoconf=-1)
app0.dump()



== Dump of cafram.NodeMap.app 140025804049328

  Infos:
    ID: 140025804049328
    Kind: NodeMap
    Ident: app
    Repr: NodeMap.140025804049328 app
    String: NodeMap:app
    MRO: NodeMap-> NodeDict-> NodeVal-> Base-> object

  Config:
  -----------------
    version: 1





Well, basically, cafram can generate the structure for you. You can change the autoconf to 0 to disable it, -1 to do all the conf, and finally an interger to mention how deep you want to create nodes.

To understand levels:
  * `-1`: Create object the whole tree
  * `0`: Do not create object automagically, all is native unless overrided
  * `1`: Create only the root object
  * `2`: Create only nodes 2 level deep
  * etc ...

It's important to note that autoconf mode only create ConfDict and ConfList. So it does not take into account leafs.


In [4]:
pprint (app0.config.get_value())
pprint (app0.config.get_children())

{'backup_prefix': None, 'backups': 3, 'create_file': True}
{'nested_dict': NodeMap.140025672256816 nested_dict}


In [5]:
# Just
app0 = NodeAuto(ident="app", payload=config, autoconf=1)
app0.dump()

app0 = NodeAuto(ident="app", payload=config, autoconf=3)
app0.dump()


== Dump of cafram.NodeMap.app 140025672698320

  Infos:
    ID: 140025672698320
    Kind: NodeMap
    Ident: app
    Repr: NodeMap.140025672698320 app
    String: NodeMap:app
    MRO: NodeMap-> NodeDict-> NodeVal-> Base-> object

  Config:
  -----------------
    version: 1
    config:
      create_file: true
      backups: 3
      backup_prefix:
      nested_dict:
        key1: val1
        key2: val2
    files:
    - name: hello
      filters:
      - author
    - name: world
    filters:
      content:
        prepend: Added at first
        append: Added at last
      author:
        name: MyName
        date: true




== Dump of cafram.NodeMap.app 140025672697024

  Infos:
    ID: 140025672697024
    Kind: NodeMap
    Ident: app
    Repr: NodeMap.140025672697024 app
    String: NodeMap:app
    MRO: NodeMap-> NodeDict-> NodeVal-> Base-> object

  Config:
  -----------------
    version: 1





But that was too easy, too magical, let's see how to build our application around this config.

## Howto - Map one class to config

We now have our configuration, let's map it to python object. For this purpose, we will reproduce the hierarchical structure we want to work with.

In [6]:
from cafram.nodes_conf import NodeMap

class MyAppV1(NodeMap):
    "This is our base application"
    
    version_min = 1
    version_max = 1
    
    def show_version(self):
        "Show version or raise Exception if not compatible"
        if self.version_min <= self.version <= self.version_max:
            print (f"Current version is {self.version}")
        else:
            raise Exception(f"Unsupported version: {self.version}")
    
    
# Instanciate our app with config
app1 = MyAppV1(ident="app", payload=config)




Then we can inspect our app, thanks to the `.dump()` method, available on every nodes:

In [7]:
app1.dump()
#pprint (app.__dict__)


== Dump of cafram.MyAppV1.app 140025672698608

  Infos:
    ID: 140025672698608
    Kind: MyAppV1
    Ident: app
    Repr: MyAppV1.140025672698608 app
    String: MyAppV1:app
    MRO: MyAppV1-> NodeMap-> NodeDict-> NodeVal-> Base-> object

  Config:
  -----------------
    version: 1
    config:
      create_file: true
      backups: 3
      backup_prefix:
      nested_dict:
        key1: val1
        key2: val2
    files:
    - name: hello
      filters:
      - author
    - name: world
    filters:
      content:
        prepend: Added at first
        append: Added at last
      author:
        name: MyName
        date: true





Then we can access to all parameters this way:

In [8]:
app1.show_version()
print ("Configuration:", app1.config)


Current version is 1
Configuration: {'create_file': True, 'backups': 3, 'backup_prefix': None, 'nested_dict': {'key1': 'val1', 'key2': 'val2'}}


However, you can't access to inner items, attributes only works on first level. That is made on purpose, this `config` dict actually represent a serializable atomic object (null/str/bool/int/dict/list). We will discover later how decompose this object in sub-object:

In [9]:
print(type(app1.files))
print(type(app1.version))
print(type(app1.config))

# So to access nested attribute
print ("Backups:", app1.config["backups"])

# Direct acces don't work
print ("FAIL: Configuration subkey: app1.config.backups" )
try:
    app1.config.backups
except AttributeError as err:
    print (err)

<class 'list'>
<class 'int'>
<class 'dict'>
Backups: 3
FAIL: Configuration subkey: app1.config.backups
'dict' object has no attribute 'backups'


This way we can serialize a whole config.

Let's check how to create sub-objects.

## Howto - Map generic classes to sub-config


We associated the method `show_version()` to our app. In a case of ConfAttr, all subvalues are available as attributes. Now we want to iterate over each files and apply a filter. For this purpose, we will map the config structue to Python classes. A first attempt is:

In [10]:
from cafram.nodes_conf import NodeMap, NodeDict, NodeList

class MyAppV2(MyAppV1):
    
    conf_struct2 = [
        {
            "key": "config",
            "cls": NodeMap,
        },
        {
            "key": "filters",
            "cls": NodeDict,
        },
        {
            "key": "files",
            "cls": NodeList,
        },
        
    ]

    
app2 = MyAppV2(ident="app", payload=config)
app2.dump()




== Dump of cafram.MyAppV2.app 140025671987104

  Infos:
    ID: 140025671987104
    Kind: MyAppV2
    Ident: app
    Repr: MyAppV2.140025671987104 app
    String: MyAppV2:app
    MRO: MyAppV2-> MyAppV1-> NodeMap-> NodeDict-> NodeVal-> Base-> object

  Config:
  -----------------
    version: 1
    config:
      create_file: true
      backups: 3
      backup_prefix:
      nested_dict:
        key1: val1
        key2: val2
    files:
    - name: hello
      filters:
      - author
    - name: world
    filters:
      content:
        prepend: Added at first
        append: Added at last
      author:
        name: MyName
        date: true





The result seems similar, but when we look deeper, child nodes has been created. On the dump, object as been added between parenthesis. Indeed, we can check internal nodes:

In [11]:
print ("\napp1 nodes:")
pprint (app1.get_children())
print ("\napp2 nodes:")
pprint (app2.get_children())

# But both config is identical
conf1 = app1.get_value()
conf2 = app2.get_value()


print ("\napp config:")
pprint (conf2)
assert conf1 == conf2


app1 nodes:
{}

app2 nodes:
{}

app config:
{'config': {'backup_prefix': None,
            'backups': 3,
            'create_file': True,
            'nested_dict': {'key1': 'val1', 'key2': 'val2'}},
 'files': [{'filters': ['author'], 'name': 'hello'}, {'name': 'world'}],
 'filters': {'author': {'date': True, 'name': 'MyName'},
             'content': {'append': 'Added at last',
                         'prepend': 'Added at first'}},
 'version': 1}


## Learn - Access to nodes and attributes

And now, if we want to access to our nested attribute:

In [12]:
# So to access nested attribute
print ("Backups1: ", app2.config.backups)
print ("Backups2: ", app2.config.get_value()['backups'])

# Item acces don't work
print ("FAIL: Configuration subkey: app2.config['backups'], because config is now a node, not a dict anymore" )

print ("Backups3: " )
try:
    app2.config["backups"]
except TypeError as err:
    print (err)

AttributeError: 'dict' object has no attribute 'backups'

## Howto - Map custom classes to sub-config


Ok, cool, we mapped json object to python object, but that does not let me add method. Wrigth, so let's create our custom classes, inherited from the basic cafram node types, we can create our new (empty) classes:

In [ ]:
class Config(NodeMap):
    pass

class Filters(NodeDict):
    pass

class Files(NodeList):
    pass


class MyAppV2(MyAppV1):
    
    conf_struct2 = [
        {
            "key": "config",
            "cls": Config,
        },
        {
            "key": "filters",
            "cls": Filters,
        },
        {
            "key": "files",
            "cls": Files,
        },
        
    ]
    
    
app2 = MyAppV2(ident="app", payload=config)
pprint (app2.get_children())
pprint (app2.get_value())

Well, we have our object assigned to python classes! Let's build our full tree:

In [ ]:
from cafram.nodes_conf import *

# App definition
#######################


# File management
# ----------------
class Config(NodeMap):
    "Application config"
    conf_struct2 = [
        {
            "key": "namespace",
            "cls": NodeVal,
        },
    ]


# Filter management
# ----------------
class Filter(NodeMap):
    "A filter configuration"
    pass

class Filters(NodeMap):
    "Filter manager"
    conf_struct2 = Filter


# File management
# ----------------
class FileFilters(NodeList):
    "Applied filters to file"
    conf_struct2 = NodeVal

class File(NodeMap):
    "File to process"

    conf_struct2 = [
        {
            "key": "name",
            "cls": NodeVal,
        },
        {
            "key": "filters",
            "cls": FileFilters,
        },
    ]

class Files(NodeList):
    "File manager"
    conf_struct2 = File

 
app2 = MyAppV2(ident="app", payload=config)



print ("\nApp nodes:")
pprint (app2.get_children())
print ("\nApp config:")
pprint (app2.get_value())

# print ("")
# pprint (app2.filters._nodes)
# print ("")
# pprint (app2.files._nodes)

## Learn - Read/Write Access to config Nodes


Let's now see how to access attributes

In [ ]:
pprint ( app2.config.get_value() )

pprint ( app2.config.backups )
pprint ( app2.config.create_file )
pprint ( app2.config.nested_dict )

# But be aware that calling an object returns itself, not it's value:
pprint ( app2.config )
app2.config.dump()

# Let's see parent object:
app2.dump()


## Howto - Assign defaults


## Howto - Assign defaults
## Learn - Navigate the tree
## Learn - Inspect the tree

